In [1]:
from wrappers.linguist_wrapper import detect_linguist
from wrappers.enry_wrapper import detect_enry

In [2]:
paths = ["example.c", "example.perl","example.scala", "example"]

In [5]:
detect_linguist(paths)

C
Perl
Scala
C


0

In [4]:
detect_enry(paths)

C
Perl
Scala
C


0